# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

## Dataset

In [ ]:
ws = Workspace.from_config()
experiment_name = 'Capstone-Project-Experiment'

experiment=Experiment(ws, experiment_name)

In [ ]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
compute_name = "hyperdrive-compute"
try:
    trainCluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_DS3_V2", max_nodes=5)
    trainCluster = ComputeTarget.create(ws, compute_name, compute_config)
trainCluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration



In [ ]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4,5), "--max_iter": choice(80,100,120,150,170,200)})

#Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='./training', compute_target=trainCluster, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, policy=early_termination_policy, primary_metric_name="Accuracy",
                                          hyperparameter_sampling=param_sampling,
                                         max_total_runs=20,
                                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [ ]:
#Submit your experiment
hyperDrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperDrive_run).show()

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])

In [ ]:
# Code below registers the best model with the information of Metrics
model = best_run.register_model(model_name='HyperDrive_HighAccuracy', model_path='outputs/', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                           'Max Iterations': best_run_metrics['Max iterations:']})